# Using more sophisticated images with Convolutional Neural Networks

I실생활의 이미지는 모양도 다르고, 비율도 다르고, 색깔도 다양하다. 이러한 것들을 분류하는 CNN을 만들어 본다.


1.   Cats and Dogs 이미지 이용
2.   개와 고양이 분류하는 Neural Network 만들기
3.   Evaluate the Training and Validation accuracy

## Explore the Example Data

 `/tmp` 컬럼에, 2000개의 이미지를 다운로드 받아서 저장한다.

**NOTE:**  2,000 개의 이미지는 캐글에서 가져왔다. ["Dogs vs. Cats" dataset](https://www.kaggle.com/c/dogs-vs-cats/data) 원래는  25,000 개의 이미지 이지만, 실습용으로 추렸음.

In [1]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

--2022-12-30 05:23:07--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.200.128, 74.125.68.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  20.7MB/s    in 3.8s    

2022-12-30 05:23:11 (17.1 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



### 압축 풀기

In [2]:
import zipfile

In [3]:
file = zipfile.ZipFile('/tmp/cats_and_dogs_filtered.zip')

In [4]:
file.extractall('/tmp')

### 데이터 억세스할 경로를 만든다.

In [5]:
base_dir = '/tmp/cats_and_dogs_filtered'

In [6]:
train_dir = '/tmp/cats_and_dogs_filtered/train'

In [7]:
test_dir = '/tmp/cats_and_dogs_filtered/validation'

### 파일명을 찍어본다.

In [8]:
# 학습용 개 사진이 저장된 디렉토리안에 있는 파일명을 확인

In [9]:
import os

In [11]:
os.listdir( train_dir + '/dogs' )

['dog.370.jpg',
 'dog.387.jpg',
 'dog.568.jpg',
 'dog.694.jpg',
 'dog.395.jpg',
 'dog.436.jpg',
 'dog.106.jpg',
 'dog.560.jpg',
 'dog.580.jpg',
 'dog.438.jpg',
 'dog.801.jpg',
 'dog.469.jpg',
 'dog.961.jpg',
 'dog.525.jpg',
 'dog.409.jpg',
 'dog.292.jpg',
 'dog.715.jpg',
 'dog.495.jpg',
 'dog.453.jpg',
 'dog.80.jpg',
 'dog.698.jpg',
 'dog.796.jpg',
 'dog.586.jpg',
 'dog.52.jpg',
 'dog.724.jpg',
 'dog.269.jpg',
 'dog.602.jpg',
 'dog.550.jpg',
 'dog.434.jpg',
 'dog.723.jpg',
 'dog.147.jpg',
 'dog.413.jpg',
 'dog.827.jpg',
 'dog.522.jpg',
 'dog.42.jpg',
 'dog.420.jpg',
 'dog.247.jpg',
 'dog.833.jpg',
 'dog.25.jpg',
 'dog.270.jpg',
 'dog.883.jpg',
 'dog.280.jpg',
 'dog.819.jpg',
 'dog.745.jpg',
 'dog.786.jpg',
 'dog.686.jpg',
 'dog.893.jpg',
 'dog.863.jpg',
 'dog.81.jpg',
 'dog.151.jpg',
 'dog.784.jpg',
 'dog.824.jpg',
 'dog.552.jpg',
 'dog.18.jpg',
 'dog.907.jpg',
 'dog.481.jpg',
 'dog.273.jpg',
 'dog.514.jpg',
 'dog.748.jpg',
 'dog.492.jpg',
 'dog.205.jpg',
 'dog.296.jpg',
 'dog.993.jpg'

### 트레이닝 이미지와 밸리데이션 이미지를, 각각 몇개씩인지 확인해 본다.

In [13]:
len( os.listdir( train_dir + '/dogs'  )  )

1000

In [14]:
len( os.listdir( train_dir + '/cats'  )  )

1000

In [15]:
len( os.listdir( test_dir + '/dogs'  )  )

500

In [16]:
len( os.listdir( test_dir + '/cats'  )  )

500

## Building a Small Model from Scratch to Get to ~72% Accuracy

이미지의 사이즈를 150x150, 칼라(rgb) 로 처리하자.


In [17]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [18]:
def build_model():
  model = Sequential()
  model.add( Conv2D(16, (3,3) , activation='relu' , input_shape=(150,150,3)  ) )
  model.add( MaxPooling2D( (2,2) , 2 ) )
  model.add( Conv2D(32, (3,3), activation='relu'))
  model.add( MaxPooling2D( (2,2) , 2 ) )
  model.add( Conv2D(64, (3,3) , activation='relu'))
  model.add( MaxPooling2D( (2,2) , 2 ) )

  model.add(Flatten())
  model.add(Dense(512, 'relu'))
  model.add(Dense(1, 'sigmoid'))
  
  model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
  return model

서머리 해보자

In [19]:
model = build_model()

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 64)       0

### RMSprop 으로 컴파일한다.

### Data Preprocessing

ImageDataGenerator 사용하기

In [21]:
from keras.preprocessing.image import ImageDataGenerator

In [25]:
train_datagen = ImageDataGenerator(rescale=1/255.0)

In [27]:
test_datagen = ImageDataGenerator(rescale=1/255.0)

In [28]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150),
                                  class_mode='binary' ,
                                  batch_size = 20)

Found 2000 images belonging to 2 classes.


In [29]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150,150),
                                 class_mode='binary' ,
                                 batch_size=20)

Found 1000 images belonging to 2 classes.


### Training
15 epochs 로 학습해 보자.


In [30]:
epoch_history = model.fit(train_generator, epochs=15, 
                          validation_data = (test_generator) , 
                          steps_per_epoch= 100 )

Epoch 1/15
100/100 [==============================] - 19s 92ms/step - loss: 0.9336 - accuracy: 0.5650 - val_loss: 0.6584 - val_accuracy: 0.6280
Epoch 2/15
100/100 [==============================] - 9s 91ms/step - loss: 0.6207 - accuracy: 0.6665 - val_loss: 0.7041 - val_accuracy: 0.6260
Epoch 3/15
100/100 [==============================] - 9s 93ms/step - loss: 0.5255 - accuracy: 0.7415 - val_loss: 0.5538 - val_accuracy: 0.7340
Epoch 4/15
100/100 [==============================] - 9s 91ms/step - loss: 0.4557 - accuracy: 0.7950 - val_loss: 0.5637 - val_accuracy: 0.7390
Epoch 5/15
100/100 [==============================] - 9s 92ms/step - loss: 0.3641 - accuracy: 0.8295 - val_loss: 0.5864 - val_accuracy: 0.7190
Epoch 6/15
100/100 [==============================] - 9s 90ms/step - loss: 0.2904 - accuracy: 0.8760 - val_loss: 0.7121 - val_accuracy: 0.7320
Epoch 7/15
100/100 [==============================] - 11s 113ms/step - loss: 0.2091 - accuracy: 0.9190 - val_loss: 0.8063 - val_accuracy: 0.6

In [ ]:
#  학습데이터 2000개 = 배치사이즈 X 스텝수 

In [31]:
model.evaluate(test_generator)

50/50 [==============================] - 3s 59ms/step - loss: 2.1401 - accuracy: 0.7160


[2.140080213546753, 0.7160000205039978]

### Running the Model

픽사베이에서 무료 사진 다운로드 받아서, 실제로 예측해 보자

In [35]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys() :
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150,150))
  x = image.img_to_array(img) / 255.0
  print(x)
  print(x.shape)

  x = np.expand_dims(x, axis = 0)

  print(x.shape)

  images = np.vstack( [x] )
  classes = model.predict( images, batch_size = 10 )
  
  print(classes)

  if classes[0] > 0.5 :
    print(fn + " is a dog")
  else :
    print(fn + " is a cat")

Saving ddd.jpg to ddd.jpg
[[[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.01176471 0.01960784 0.00784314]
  [0.01176471 0.01960784 0.00784314]
  [0.01176471 0.01960784 0.00784314]]

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.01176471 0.01960784 0.00784314]
  [0.01176471 0.01960784 0.00784314]
  [0.01176471 0.01960784 0.00784314]]

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.01176471 0.01960784 0.00784314]
  [0.01176471 0.01960784 0.00784314]
  [0.01176471 0.01960784 0.00784314]]

 ...

 [[0.5529412  0.56078434 0.3137255 ]
  [0.54509807 0.5647059  0.3137255 ]
  [0.54509807 0.5647059  0.30588236]
  ...
  [0.52156866 0.5568628  0.3019608 ]
  [0.5058824  0.54901963 0.2901961 ]
  [0.5058824  0.54901963 0.2901961 ]]

 [[0.5568628  0.5647059  0.31764707]
  [0.549019

### Evaluating Accuracy and Loss for the Model

training/validation accuracy 와 loss 를 차트로 시각화 한다.

 **overfitting** 을 확인해 보자